<a href="https://colab.research.google.com/github/KanakoShImoyama/202007-kanako/blob/master/pytorch_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 今後の方針
1. algorith.pyをoptimizer フォルダに作る
1. if \_\_name\_\_ == '\_\_main\_\_': をimdb() だけにする
1. main.py と同じフォルダに入った状態で python main.py

In [1]:
# import os
# assert os.environ['COLAB_TPU_ADDR'], 'Make sure to select TPU from Edit > Notebook settings > Hardware accelerator'

In [2]:
!pip install torch<=1.2.0
!pip install torchtext==0.5

# VERSION = "20200325"  #@param ["1.5" , "20200325", "nightly"]
# !curl https://raw.githubusercontent.com/pytorch/xla/master/contrib/scripts/env-setup.py -o pytorch-xla-env-setup.py
# !python pytorch-xla-env-setup.py --version $VERSION

/bin/bash: =1.2.0: No such file or directory
     |████████████████████████████████| 81kB 8.4MB/s 
     |████████████████████████████████| 1.2MB 12.4MB/s 
  Found existing installation: torchtext 0.3.1
    Uninstalling torchtext-0.3.1:
      Successfully uninstalled torchtext-0.3.1


In [3]:
import numpy as np
import pandas as pd

In [4]:
import math
import torch
from torch.optim.optimizer import Optimizer


class SGradients(Optimizer):

    def __init__(self, params, betas=(0.9, 0.999), eps=1e-8,
                 weight_decay=0, amsgrad=False, alpha=1e-3, beta=0.9, diminish=False):
        if not 0.0 <= eps:
            raise ValueError("Invalid epsilon value: {}".format(eps))
        if not 0.0 <= betas[0] < 1.0:
            raise ValueError("Invalid beta parameter at index 0: {}".format(betas[0]))
        if not 0.0 <= betas[1] < 1.0:
            raise ValueError("Invalid beta parameter at index 1: {}".format(betas[1]))
        if not 0.0 <= weight_decay:
            raise ValueError("Invalid weight_decay value: {}".format(weight_decay))
        defaults = dict(betas=betas, eps=eps,
                        weight_decay=weight_decay, amsgrad=amsgrad, alpha=alpha, beta=beta, diminish=diminish)
        super().__init__(params, defaults)

    def __setstate__(self, state):
        super().__setstate__(state)
        for group in self.param_groups:
            group.setdefault('amsgrad', False)

    @torch.no_grad()
    def step(self, closure=None):
        """Performs a single optimization step.

        Arguments:
            closure (callable, optional): A closure that reevaluates the model
                and returns the loss.
        """
        loss = None
        if closure is not None:
            with torch.enable_grad():
                loss = closure()

        for group in self.param_groups:
            for x in group['params']:
                if x.grad is None:
                    continue
                grad = x.grad
                if grad.is_sparse:
                    raise RuntimeError('Adam does not support sparse gradients, please consider SparseAdam instead')
                amsgrad = group['amsgrad']

                state = self.state[x]

                # State initialization
                if len(state) == 0:
                    state['n'] = 0
                    # Exponential moving average of gradient values
                    state['m'] = torch.zeros_like(x, memory_format=torch.preserve_format)
                    # Exponential moving average of squared gradient values
                    state['v'] = torch.zeros_like(x, memory_format=torch.preserve_format)
                    ### v_hatを初期化
                    state['v_hat'] = torch.zeros_like(x, memory_format=torch.preserve_format)
                    ### alphaを初期化
                    state['alpha'] = group['alpha']
                    ### betaを初期化
                    state['beta'] = group['beta']

                m, v, v_hat = state['m'], state['v'], state['v_hat']

                if group['diminish'] and state['n'] > 0:
                  state['alpha'] /= state['n'] ** 0.5
                  try:
                    state['beta'] /= 2 ** state['n']
                  except OverflowError:
                    state['beta'] = 0
                  
                beta_hat, beta_bar = group['betas']

                if not amsgrad:
                    bias_correction_hat = 1 - beta_hat ** (state['n'] + 1)
                    bias_correction_bar = 1 - beta_bar ** (state['n'] + 1) 

                if group['weight_decay'] != 0:
                    grad = grad.add(x, alpha=group['weight_decay'])

                # Decay the first and second moment running average coefficient
                m.mul_(state['beta']).add_(grad, alpha=1 - state['beta'])
                ####変更部分addmul_(最初のgrad)####
                v.mul_(beta_bar).addcmul_(grad**2 ,grad**2 , value=1 - beta_bar)
                if amsgrad:
                    v_bar = v
                    torch.max(v_hat, v_bar, out=v_hat)
                    step_size = state['alpha']
                else:
                    v_bar = v / bias_correction_bar
                    torch.max(v_hat, v_bar, out=v_hat)
                    step_size = state['alpha'] / bias_correction_hat
                ####変更部分＃＃＃＃＃＃＃＃
                h = v_hat.sqrt().sqrt().add_(group['eps'])
                #h = v_hat.sqrt().add_(group['eps'])

                x.addcdiv_(m, h, value=-step_size)

                state['n'] += 1
        return loss

# 新しいセクション

In [5]:
import torch
import torchtext
from torchtext.datasets import text_classification
#import torch_xla.core.xla_model as xm

NGRAMS = 2
import os
if not os.path.isdir('./.data'):
	os.mkdir('./.data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = xm.xla_device()

ag_news_csv.tar.gz: 11.8MB [00:00, 60.9MB/s]
120000lines [00:06, 17669.36lines/s]
120000lines [00:15, 7675.45lines/s]
7600lines [00:00, 7790.07lines/s]


In [6]:
device

device(type='cuda')

In [7]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()
        
    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [8]:
VOCAB_SIZE = len(train_dataset.get_vocab())
EMBED_DIM = 32
NUN_CLASS = len(train_dataset.get_labels())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

In [9]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)
    
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [10]:
from torch.utils.data import DataLoader

def train_func(sub_train_):

    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        g_k_1 = torch.empty(1)
        d_k_1 = np.nan
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    # scheduler.step()
    
    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

## 実験本体

In [ ]:
import time
from torch.utils.data.dataset import random_split
from torch import optim

N_EPOCHS = 200
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = SGradients(model.parameters(), alpha=1e-2, beta=1e-2, diminish=False, amsgrad=True)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

train_len = int(len(train_dataset) * 0.95)
print("train:", train_len, ", valid:", len(train_dataset) - train_len, ", test:", len(test_dataset))

sub_train_, sub_valid_ = random_split(train_dataset, [train_len, len(train_dataset) - train_len])

train_loss_list = [] # 保存したいもののリストを作る
train_acc_list = []
valid_loss_list = []
valid_acc_list = []
secs_list = []
for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    secs_list.append(secs) # 時間保存

    mins = secs / 60
    secs = secs % 60

    train_loss_list.append(train_loss) # リストに保存する
    train_acc_list.append(train_acc)
    valid_loss_list.append(valid_loss)
    valid_acc_list.append(valid_acc)

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

result_dict = dict(
    train_loss = train_loss_list, # 名前=リストを追加していく
    train_acc = train_acc_list,
    valid_loss = valid_loss_list,
    valid_acc = valid_acc_list,
    time=secs_list,
)
csv_name = 'result.csv'
df_result = pd.DataFrame(result_dict).to_csv(csv_name, index=False, encoding='utf-8')

train: 114000 , valid: 6000 , test: 7600
Epoch: 1  | time in 2 minutes, 57 seconds
	Loss: 0.0178(train)	|	Acc: 90.7%(train)
	Loss: 0.0001(valid)	|	Acc: 92.4%(valid)
Epoch: 2  | time in 2 minutes, 55 seconds
	Loss: 0.0045(train)	|	Acc: 97.8%(train)
	Loss: 0.0001(valid)	|	Acc: 91.8%(valid)
Epoch: 3  | time in 2 minutes, 55 seconds
	Loss: 0.0014(train)	|	Acc: 99.4%(train)
	Loss: 0.0002(valid)	|	Acc: 91.8%(valid)
Epoch: 4  | time in 2 minutes, 55 seconds
	Loss: 0.0008(train)	|	Acc: 99.6%(train)
	Loss: 0.0001(valid)	|	Acc: 91.9%(valid)
Epoch: 5  | time in 2 minutes, 55 seconds
	Loss: 0.0005(train)	|	Acc: 99.7%(train)
	Loss: 0.0002(valid)	|	Acc: 92.0%(valid)
Epoch: 6  | time in 2 minutes, 55 seconds
	Loss: 0.0004(train)	|	Acc: 99.8%(train)
	Loss: 0.0001(valid)	|	Acc: 91.9%(valid)
Epoch: 7  | time in 2 minutes, 55 seconds
	Loss: 0.0004(train)	|	Acc: 99.8%(train)
	Loss: 0.0001(valid)	|	Acc: 92.0%(valid)
Epoch: 8  | time in 2 minutes, 55 seconds
	Loss: 0.0003(train)	|	Acc: 99.8%(train)
	Loss: 0

# 新しいセクション

In [ ]:
df_result = pd.DataFrame(dict(
    time=[10, 13, 15],
    train_loss=[.5, .6, .3]
))

In [ ]:
df_result.to_csv('result.csv')

In [ ]:
import re
from torchtext.data.utils import ngrams_iterator
from torchtext.data.utils import get_tokenizer

ag_news_label = {1 : "World",
                 2 : "Sports",
                 3 : "Business",
                 4 : "Sci/Tec"}

def predict(text, model, vocab, ngrams):
    tokenizer = get_tokenizer("basic_english")
    with torch.no_grad():
        text = torch.tensor([vocab[token]
                            for token in ngrams_iterator(tokenizer(text), ngrams)])
        output = model(text, torch.tensor([0]))
        return output.argmax(1).item() + 1

ex_text_str = "MEMPHIS, Tenn. – Four days ago, Jon Rahm was \
    enduring the season’s worst weather conditions on Sunday at The \
    Open on his way to a closing 75 at Royal Portrush, which \
    considering the wind and the rain was a respectable showing. \
    Thursday’s first round at the WGC-FedEx St. Jude Invitational \
    was another story. With temperatures in the mid-80s and hardly any \
    wind, the Spaniard was 13 strokes better in a flawless round. \
    Thanks to his best putting performance on the PGA Tour, Rahm \
    finished with an 8-under 62 for a three-stroke lead, which \
    was even more impressive considering he’d never played the \
    front nine at TPC Southwind."

vocab = train_dataset.get_vocab()
model = model.to("cpu")

print("This is a %s news" %ag_news_label[predict(ex_text_str, model, vocab, 2)])